In [1]:
import os
import dotenv
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
university_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world.
"""

## Create a vector store

In [3]:
from openai import OpenAI
oai_client = OpenAI()

oai_client.embeddings.create(
        model="text-embedding-ada-002",
        input=university_info
    )

CreateEmbeddingResponse(data=[Embedding(embedding=[0.008575839921832085, -0.0037293159402906895, -0.005757429171353579, -0.023524843156337738, -0.02051599882543087, -0.008639317005872726, -0.005951036233454943, 0.0034722310956567526, -0.0015766278374940157, -0.022559981793165207, -0.028184106573462486, 0.0046878294087946415, 0.01776106469333172, -0.01277171727269888, -0.0032151464838534594, -0.01962731033563614, 0.011762421578168869, -0.03092634491622448, -0.0073380242101848125, -0.01708820089697838, -0.026457512751221657, -0.014803003519773483, -0.00041022474761120975, -0.004764002747833729, -0.011387902311980724, 0.02253459021449089, 0.003088190918788314, -0.01609794981777668, 0.024692831560969353, -0.010378606617450714, -0.00713489530608058, -0.009756525047123432, -0.03953392431139946, 0.010016784071922302, -0.00909635704010725, 0.004814784973859787, -0.007306285202503204, 0.006195425055921078, 0.0094010503962636, -0.00739515433087945, 0.009832698851823807, 0.008233060128986835, -0.

In [4]:
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

embedding_function = OpenAIEmbeddingFunction(api_key=os.environ.get('OPENAI_API_KEY'),
                                             model_name="text-embedding-ada-002")


chroma_client = chromadb.Client()
vector_store = chroma_client.get_or_create_collection(name="Universities",
                                                      embedding_function=embedding_function)

In [5]:
vector_store.add("uni_info", documents=university_info)

## Building a RAG

In [6]:
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
tru = Tru()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [7]:
class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vector_store.query(
        query_texts=query,
        n_results=2
    )
        return results['documents'][0]

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        completion = oai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=0,
        messages=
        [
            {"role": "user",
            "content": 
            f"We have provided context information below. \n"
            f"---------------------\n"
            f"{context_str}"
            f"\n---------------------\n"
            f"Given this information, please answer the question: {query}"
            }
        ]
        ).choices[0].message.content
        return completion

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        return completion

rag = RAG_from_scratch()

## Setup feedback functions

In [8]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI as fOpenAI

import numpy as np

# Initialize provider class
fopenai = fOpenAI()

grounded = Groundedness(groundedness_provider=fopenai)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = (
    Feedback(fopenai.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fopenai.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.app.retrieve.args.query .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input statement will be set to __record__.app.retrieve.rets.collect() .


## Wraping RAG custon app with list of feedbacks

In [9]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = 'RAG v1',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

## Running the app

In [10]:
with tru_rag as recording:
    rag.query("When was the University of Washington founded?")

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


In [11]:
tru.get_leaderboard(app_ids=["RAG v1"])

,Answer Relevance,Groundedness,Context Relevance,latency,total_cost
app_id,,,,,
RAG v1,1.0,1.0,0.9,3.5,0.000203


In [12]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.52.239:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>